In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# ----------------------------
# 1. Load Models
# ----------------------------
qa_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# ----------------------------
# 2. Build Knowledge Base
# ----------------------------
def load_knowledge(file_path="sample.txt"):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    # split into sentences (basic, can improve with regex/chunking)
    chunks = [c.strip() for c in text.split("\n") if c.strip()]
    return chunks

knowledge_chunks = load_knowledge("sample.txt")
chunk_embeddings = embedder.encode(knowledge_chunks, convert_to_numpy=True)

# Create FAISS index
d = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(chunk_embeddings)

# ----------------------------
# 3. Conversational Memory
# ----------------------------
chat_history = []  # list of (user, bot)

def retrieve_context(query, k=2):
    query_emb = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(query_emb, k)
    retrieved = [knowledge_chunks[i] for i in I[0] if i < len(knowledge_chunks)]
    return "\n".join(retrieved)

def build_prompt(user_query, context):
    history_text = ""
    for u, b in chat_history[-3:]:
        history_text += f"User: {u}\nBot: {b}\n"
    prompt = f"""
You are a helpful assistant.

Conversation so far:
{history_text}

Knowledge Context:
{context}

Now answer the new question:
User: {user_query}
Bot:"""
    return prompt

def generate_answer(user_query):
    # 1. Retrieve
    context = retrieve_context(user_query)

    # 2. Build Prompt
    prompt = build_prompt(user_query, context)

    # 3. Generate
    inputs = qa_tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = qa_model.generate(**inputs, max_length=150)
    answer = qa_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 4. Save to memory
    chat_history.append((user_query, answer))
    return answer

# ----------------------------
# 4. Example Conversation
# ----------------------------
print("User: What is Python?")
print("Bot:", generate_answer("What is Python?"))

print("\nUser: And what is Java?")
print("Bot:", generate_answer("And what is Java?"))

print("\nUser: Compare Python and Java in short.")
print("Bot:", generate_answer("Compare Python and Java in short."))


User: What is Python?
Bot: a high-level programming language

User: And what is Java?
Bot: a versatile, object-oriented programming language

User: Compare Python and Java in short.
Bot: Python is dynamically typed and concise, while Java is statically typed and verbose.
